## XGBoost

In [610]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

#modeling
import xgboost as xgb

In [513]:
train = pd.read_csv('data/train_preProcess.csv')
test = pd.read_csv('data/test_preProcess.csv')

In [515]:
train_y = train.SalePrice
train.drop(['SalePrice'], axis=1, inplace=True)

train.drop(['Id'], axis=1, inplace=True)

test_id = test['Id']
test.drop(['Id'], axis=1, inplace=True)

df = pd.concat([train, test], axis= 0)

In [398]:
#data dimension
df.shape

(2916, 79)

### Randomized Grid Search

In [577]:
gbm = xgb.XGBRegressor(nthread=4, random_state=123)

gbm_param_grid = {
    'learning_rate': np.arange(0, 0.15, 0.01),
    'gamma': np.arange(0.001, 0.025, 0.001),
    'max_depth': np.arange(2, 10, 1),
    'n_estimators': np.arange(15, 2000, 50),
    "colsample_bytree": np.arange(0.05, 0.15, 0.01)
}

randomized_reg = RandomizedSearchCV(estimator=gbm, param_distributions= gbm_param_grid, n_iter=100, cv=10,
                                    scoring="neg_mean_squared_error", verbose = 20)

In [578]:
randomized_reg.fit(train, train_y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] colsample_bytree=0.07, learning_rate=0.05, n_estimators=1165, max_depth=9, gamma=0.001 
[CV]  colsample_bytree=0.07, learning_rate=0.05, n_estimators=1165, max_depth=9, gamma=0.001, score=-0.015099886651343321, total=   0.5s
[CV] colsample_bytree=0.07, learning_rate=0.05, n_estimators=1165, max_depth=9, gamma=0.001 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  colsample_bytree=0.07, learning_rate=0.05, n_estimators=1165, max_depth=9, gamma=0.001, score=-0.012681332990380888, total=   0.5s
[CV] colsample_bytree=0.07, learning_rate=0.05, n_estimators=1165, max_depth=9, gamma=0.001 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s


[CV]  colsample_bytree=0.07, learning_rate=0.05, n_estimators=1165, max_depth=9, gamma=0.001, score=-0.015320000037149948, total=   0.5s
[CV] colsample_bytree=0.07, learning_rate=0.05, n_estimators=1165, max_depth=9, gamma=0.001 


...

[CV]  colsample_bytree=0.11, learning_rate=0.13, n_estimators=1465, max_depth=5, gamma=0.02, score=-0.017981353928274914, total=   0.4s
[CV] colsample_bytree=0.11, learning_rate=0.13, n_estimators=1465, max_depth=5, gamma=0.02 
[CV]  colsample_bytree=0.11, learning_rate=0.13, n_estimators=1465, max_depth=5, gamma=0.02, score=-0.025335083518297146, total=   0.5s
[CV] colsample_bytree=0.11, learning_rate=0.13, n_estimators=1465, max_depth=5, gamma=0.02 
[CV]  colsample_bytree=0.11, learning_rate=0.13, n_estimators=1465, max_depth=5, gamma=0.02, score=-0.011292679877311392, total=   0.4s
[CV] colsample_bytree=0.11, learning_rate=0.13, n_estimators=1465, max_depth=5, gamma=0.02 
[CV]  colsample_bytree=0.11, learning_rate=0.13, n_estimators=1465, max_depth=5, gamma=0.02, score=-0.0159759912141949, total=   0.4s
[CV] colsample_bytree=0.11, learning_rate=0.13, n_estimators=1465, max_depth=5, gamma=0.02 
[CV]  colsample_bytree=0.11, learning_rate=0.13, n_estimators=1465, max_depth=5, gamma=0.0

[CV]  colsample_bytree=0.06, learning_rate=0.12, n_estimators=415, max_depth=3, gamma=0.007, score=-0.018655694805192947, total=   0.0s
[CV] colsample_bytree=0.14, learning_rate=0.04, n_estimators=165, max_depth=7, gamma=0.015 
[CV]  colsample_bytree=0.14, learning_rate=0.04, n_estimators=165, max_depth=7, gamma=0.015, score=-0.014822351742776562, total=   0.0s
[CV] colsample_bytree=0.14, learning_rate=0.04, n_estimators=165, max_depth=7, gamma=0.015 
[CV]  colsample_bytree=0.14, learning_rate=0.04, n_estimators=165, max_depth=7, gamma=0.015, score=-0.01532251856599784, total=   0.0s
[CV] colsample_bytree=0.14, learning_rate=0.04, n_estimators=165, max_depth=7, gamma=0.015 
[CV]  colsample_bytree=0.14, learning_rate=0.04, n_estimators=165, max_depth=7, gamma=0.015, score=-0.016173125644963125, total=   0.0s
[CV] colsample_bytree=0.14, learning_rate=0.04, n_estimators=165, max_depth=7, gamma=0.015 
[CV]  colsample_bytree=0.14, learning_rate=0.04, n_estimators=165, max_depth=7, gamma=0.0

[CV]  colsample_bytree=0.07, learning_rate=0.04, n_estimators=815, max_depth=3, gamma=0.005, score=-0.011575419816757544, total=   0.1s
[CV] colsample_bytree=0.07, learning_rate=0.04, n_estimators=815, max_depth=3, gamma=0.005 
[CV]  colsample_bytree=0.07, learning_rate=0.04, n_estimators=815, max_depth=3, gamma=0.005, score=-0.014217988584560238, total=   0.1s
[CV] colsample_bytree=0.07, learning_rate=0.04, n_estimators=815, max_depth=3, gamma=0.005 
[CV]  colsample_bytree=0.07, learning_rate=0.04, n_estimators=815, max_depth=3, gamma=0.005, score=-0.009673278128076881, total=   0.1s
[CV] colsample_bytree=0.07, learning_rate=0.04, n_estimators=815, max_depth=3, gamma=0.005 
[CV]  colsample_bytree=0.07, learning_rate=0.04, n_estimators=815, max_depth=3, gamma=0.005, score=-0.008634002335386203, total=   0.1s
[CV] colsample_bytree=0.07, learning_rate=0.04, n_estimators=815, max_depth=3, gamma=0.005 
[CV]  colsample_bytree=0.07, learning_rate=0.04, n_estimators=815, max_depth=3, gamma=0.

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  6.4min finished


RandomizedSearchCV(cv=10, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=4, objective='reg:linear', random_state=123,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'colsample_bytree': array([ 0.05,  0.06,  0.07,  0.08,  0.09,  0.1 ,  0.11,  0.12,  0.13,  0.14]), 'learning_rate': array([ 0.  ,  0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,
        0.09,  0.1 ,  0.11,  0.12,  0.13,  0.14]), 'n_estimators': array([  15,   65,  115,  ...13,  0.014,  0.015,  0.016,
        0.017,  0.018,  0.019,  0.02 ,  0.021,  0.022,  0.023,  0.024])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          r

In [579]:
np.sqrt(np.abs(randomized_reg.best_score_))

0.11609401043655639

In [607]:
randomized_reg.best_params_

{'colsample_bytree': 0.060000000000000005,
 'gamma': 0.017000000000000001,
 'learning_rate': 0.029999999999999999,
 'max_depth': 3,
 'n_estimators': 1315}

In [599]:
params = randomized_reg.best_params_

In [541]:
num_train = len(train)
num_val = int(num_train * 0.2)

x_train_all = train
x_train = train[:-num_val]
x_val = train[num_train-num_val:]

ylog_train = train_y[:-num_val]
ylog_val = train_y[num_train-num_val:]

In [542]:
df_columns = train.columns

In [543]:
dtrin_all = xgb.DMatrix(x_train_all, train_y, feature_names= df_columns)
dtrain = xgb.DMatrix(x_train, ylog_train, feature_names= df_columns)
dval = xgb.DMatrix(x_val, ylog_val, feature_names= df_columns)
dtest = xgb.DMatrix(test, feature_names=df_columns)

In [609]:
xgb_model = xgb.train(params, dtrain, num_boost_round= 10000, evals=[(dval, 'val')],
                      early_stopping_rounds = 500, verbose_eval=100)

[0]	val-rmse:11.1944
Will train until val-rmse hasn't improved in 500 rounds.
[100]	val-rmse:0.58409
[200]	val-rmse:0.151682
[300]	val-rmse:0.132253
[400]	val-rmse:0.125653
[500]	val-rmse:0.122912
[600]	val-rmse:0.121325
[700]	val-rmse:0.120151
[800]	val-rmse:0.119531
[900]	val-rmse:0.11866
[1000]	val-rmse:0.118058
[1100]	val-rmse:0.117647
[1200]	val-rmse:0.117253
[1300]	val-rmse:0.117023
[1400]	val-rmse:0.116553
[1500]	val-rmse:0.116405
[1600]	val-rmse:0.116161
[1700]	val-rmse:0.115958
[1800]	val-rmse:0.115726
[1900]	val-rmse:0.115653
[2000]	val-rmse:0.115475
[2100]	val-rmse:0.11518
[2200]	val-rmse:0.115084
[2300]	val-rmse:0.114937
[2400]	val-rmse:0.114887
[2500]	val-rmse:0.114881
[2600]	val-rmse:0.114874
[2700]	val-rmse:0.114812
[2800]	val-rmse:0.114728
[2900]	val-rmse:0.114724
[3000]	val-rmse:0.114649
[3100]	val-rmse:0.114619
[3200]	val-rmse:0.114573
[3300]	val-rmse:0.114528
[3400]	val-rmse:0.114557
[3500]	val-rmse:0.114497
[3600]	val-rmse:0.114477
[3700]	val-rmse:0.114468
[3800]	va

In [603]:
num_boost_round = xgb_model.best_iteration
final_model = xgb.train(dict(params, silent =0), dtrain, num_boost_round=num_boost_round)

In [604]:
ylog_pred = final_model.predict(dtest)
y_pred = np.exp(ylog_pred) - 1

In [606]:
df_sub = pd.DataFrame({'id': test_id, "SalePrice":y_pred})
df_sub.to_csv('final_out.csv', index=False)